In [98]:
import pandas as pd 
import solid 
from math import * 

In [3]:
data = pd.read_csv('https://data.cityofchicago.org/api/views/9m5u-3f29/rows.csv?accessType=DOWNLOAD')

In [26]:
data.describe()

,Average Days to Complete Graffiti Removal,Total Completed Requests,Median Days to Complete Graffiti Removal Request,Target Response Time (Days)
count,28.000000,28.000000,27.000000,2
mean,7.456429,2099.785714,2.962963,10
std,2.088088,340.938830,1.192331,0
min,3.100000,1430.000000,1.000000,10
25%,6.047500,1927.500000,2.000000,10
50%,7.325000,2160.000000,3.000000,10
75%,9.175000,2309.000000,4.000000,10
max,11.570000,2706.000000,5.000000,10


In [14]:
polygon =solid.polygon(([1,2],[2,3],[1,3],[2,2]))

In [20]:
from solid.utils import *
extrude_along_path

<function solid.utils.extrude_along_path>

In [21]:
def path():
    return [Point3(0,0,0), Point3(0,0,1)]

In [24]:
shape =extrude_along_path(shape_pts=[Point3(1,2,0),Point3(2,3,0),Point3(1,3,0),Point3(2,2,0)], path_pts=path())



polyhedron(points = [[0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, 0.0000000000, 1.0000000000], [0.0000000000, 0.0000000000, 1.0000000000], [0.0000000000, 0.0000000000, 1.0000000000], [0.0000000000, 0.0000000000, 1.0000000000]], triangles = [[0, 4, 1], [1, 4, 5], [1, 5, 2], [2, 5, 6], [2, 6, 3], [3, 6, 7], [0, 3, 7], [0, 7, 4], [0, 1, 2], [0, 2, 3], [4, 6, 5], [4, 7, 6]]);


In [16]:
from solid import *
d = difference()(
    cube(10),
    sphere(15)
)
print(scad_render(d))



difference() {
	cube(size = 10);
	sphere(r = 15);
}


## Get data

In [99]:
a = []
for i in range(1,13):
    monthly = pd.read_csv('https://www.ncdc.noaa.gov/cag/time-series/global/globe/land_ocean/1/{0}/1880-2015.csv'.format(i), header=3)
    monthly['month'] = i
    a.append(monthly)
total = pd.concat(a)

In [117]:
def angle(month):
    return (month/12.)*2*pi

In [163]:
def radius(data,max=100,min=10):
    return (data - min_val)*(max-min)/(max_val-min_val) + min

In [170]:
max_val, min_val = total['Value'].max(), total['Value'].min()
segment_height = 2

In [171]:
def year_path( values, months, height= 10):
    shape = [[radius(d)*sin(angle(month-1)), radius(d)*cos(angle(month-1))] for month,d in zip(months, values)]
    return linear_extrude(height=height)(polygon(points=shape))

In [172]:
segments = []
for offset, year  in enumerate(range(total['Year'].min(), total['Year'].max())):
    data = total[total['Year']==year].sort(columns='month', ascending=True)
    segments.append(up(segment_height*offset)(year_path(data['Value'], data['month'],height=segment_height)))
final = union()(segments)

In [173]:
with open('sculpture.scad', 'w') as f:
    f.write(scad_render(final).replace('triangles','faces'))

In [174]:
polygon??